#개와 사자의 이미지로 분류및 두개의 이미지 얼굴인식

In [ ]:
import os
from glob import glob

##크롤링한 이미지 넘버링 정렬및 이미지 비율 확인

In [ ]:
path = "/content/drive/MyDrive/animals"
#glob은 파일을 리스트형태로 변환해주기위해 사용.
dog = glob(path+"/dog"+'/*')
lion = glob(path +"/lion"+'/*')


In [ ]:
#이미지파일 이름 정렬

def rename(files):
    if 'dog' in files[0]:
        for i,f in enumerate(files):
            os.rename(f, os.path.join(path+"/dog", 'dog_' +'{0:03d}.jpg'.format(i)))
        dog = glob(path+"/dog"+'/*')
        print("dog {}번째 이미지까지 성공".format(i+1))

    elif 'lion' in files[0]:
        for i,f in enumerate(files):
            os.rename(f, os.path.join(path+"/lion", 'lion_' +'{0:03d}.jpg'.format(i)))
        lion = glob(path+"/lion"+'/*')
        print("lion {}번째 이미지까지 성공".format(i+1))

In [ ]:
#이름 정렬하기
#rename(dog)
#rename(lion)
#한번만 작동시켜야함 두번이상 작동하면 이상하게 뒤섞임.

In [ ]:
#이미지 리스트 정렬하기
dog = sorted(glob(path+"/dog"+'/*'))
lion = sorted(glob(path+"/lion"+'/*'))

print('dog 이미지 개수: {}\nlion 이미지 개수: {}'.format(len(dog),len(lion)))

In [ ]:
#이미지 총갯수 및 비율
image_sum = len(dog)+ len(lion)
print('전체 이미지 개수 : {}\n\ndog 이미지 비율: {:.2f}%\nlion 이미지 비율:{:.2f}%'.format(image_sum,100*len(dog)/image_sum,100*len(lion)/image_sum)) 

In [ ]:
#베이스라인 모델을 만들기위해서 이미지 데이터 합침
data_list = dog + lion
print(data_list)

##크롤링이미지 전처리전 시각화

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import cv2
import numpy as np
%matplotlib inline

In [ ]:
#이미지를 읽게 만들어주는 함수 정의
def read_img(file_path):
    img_arr = cv2.imread(file_path)
    return cv2.cvtColor(np.ascontiguousarray(img_arr),cv2.COLOR_BGR2RGB)
#cv2.COLOR_BGR2RGB는 bgr 컬러형태의 이미지를 rgb 컬러형태로 변환해줌.
#이미지 출력할때 bgr형태의 데이터로는 출력이 안된다. 옵션에 rgb형태의 데이터로 바꾼후 넣어줘야한다함.

In [ ]:
#각 각의 이미지 폴더에서 랜덤으로 이미지 시각화 시키기.
import random

img_arrs = []
img_num = range(0,300)
#random.sample은 100개의 이미지가 있으면 섞어서 뽑아준다는것.
for i in random.sample(img_num,5):

    img_arrs.append(read_img(dog[i]))
    img_arrs.append(read_img(lion[i]))

In [ ]:
print('총 {}개의 이미지'.format(len(img_arrs)))

In [ ]:
rows = 3 #3개씩 샘플링해서.
columns= 2

#출력 이미지 설정 figsize설정시에 columns,rows 곱해주는 값을 다르게 해주면 모양이 이상하게 나옴.
fig, axes = plt.subplots(nrows=rows,ncols=columns,figsize=(columns*3,rows*3))

#설정한대로 이미지 출력
for number in range(1, rows*columns+1):
    
    fig.add_subplot(rows, columns, number)# 원하는 위치 번호를 입력해줌.

    idx = number - 1 #인덱스값

    plt.imshow(img_arrs[idx], aspect='auto')# aspect를 auto로해서 픽셀형식으로 출력이 된다함.
    plt.xlabel(f'{img_arrs[idx].shape}', fontsize=10)

fig.tight_layout()

# 이미지 재목 설정해주기
cols = ['dog','lion']

for folder_idx, ax in enumerate(axes[0]):
    ax.set_title(cols[folder_idx])

# xtick,ytick 즉 x축 y축의 이미지안의 눈금값을 없애줌
for idx, ax in enumerate(axes.flat):
    ax.set_xticks([])
    ax.set_yticks([])

## 이미지 전처리 진행

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import text_to_word_sequence
#이미지 전처리 함수화 진행
image_height = 256
image_width = 256
channel_n = 3
batch_size = image_sum

def make_file(image_height,image_width,channel_n,batch_size):
    label = []
    images = np.zeros((batch_size, image_height,image_width,channel_n))
    for n,path in enumerate(data_list[:batch_size]):
        
    #라벨
        token = text_to_word_sequence(data_list[n])
        label.append(token[0])
    #데이터 256,256,3형태로 변환   
        image = cv2.imread(data_list[n])
        image = cv2.resize(image,(image_height,image_width))/255
        images[n,:,:,:] = image

    label = np.array(label)

    return (label,images)


In [ ]:
#만든 전처리함수를 이용해서 라벨과 이미지 데이터셋 가져오기
label, images = make_file(image_height,image_width,channel_n,batch_size)


In [ ]:
#데이터와 라벨 구조 확인
print(images.shape,label.shape)#잘 나옵니다,

In [ ]:
#data와 label 인코딩
from sklearn.preprocessing import LabelEncoder
items = label
encoder = LabelEncoder()
encoder.fit(items)
label = encoder.transform(items)

## 베이스 모델 만들기


In [ ]:
#data train,test 분할
from sklearn.model_selection import train_test_split
x = images
y = label

X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)


In [ ]:
print(X_train.shape,X_test.shape)#??잘 나눠진것 같긴하다

In [ ]:
from tensorflow.keras.models import Sequential, Model 
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
#베이스 모델은 이미지분류의 기본모델인 CNN을 사용할것입니다.
model = Sequential()
model.add(Conv2D(32, (3,3),activation='relu',input_shape=(image_height, image_width, channel_n)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
base_model = model.fit(X_train,y_train, validation_data=(X_test,y_test),batch_size=256,epochs=2)

In [ ]:
#베이스라인 모델 시각화
#??엄청나게 좋은 성능을 보이고 있다. 데이터가 적은탓인거같기도함.
plt.figure(figsize=(12,4))
plt.subplot(1, 2, 1)
plt.plot(base_model.history['accuracy'])
plt.plot(base_model.history['val_accuracy'])
plt.title('model accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['train', 'validation'])

plt.subplot(1, 2, 2)
plt.plot(base_model.history['loss'])
plt.plot(base_model.history['val_loss'])
plt.title('model loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'validation'])

## 저 베이스라인을 토대로 얼굴인식 모델 만들기
